#Model 1D CNN Dense

A stand-alone model intended to be imported as a .py file.

This version being used for PSG-Audio work.  Called by Evaluate_Model_PSG-Audio.ipynb.

Author:  [Lee B. Hinkle](https://userweb.cs.txstate.edu/~lbh31/), [IMICS Lab](https://imics.wp.txstate.edu/), Texas State University, 2023

<a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-sa/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/">Creative Commons Attribution-ShareAlike 4.0 International License</a>.

TODO:
* 

# Imports

In [ ]:
import os
import shutil
import time
from datetime import datetime
from datetime import date
from datetime import timedelta # for calculating training time
from random import randrange # for plotting samples
import numpy as np
import matplotlib.pyplot as plt # for plotting training curves
from tabulate import tabulate # for verbose tables, showing data
import tensorflow as tf
from tensorflow import keras #added to save model
from tensorflow.keras import layers #format matches MNIST example
from tensorflow.keras.callbacks import EarlyStopping
#imports for computing and displaying output metrics
# import seaborn as sns
# import pandas as pd
# from sklearn.metrics import classification_report
# from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support
# from sklearn.preprocessing import LabelEncoder
# from sklearn.preprocessing import OneHotEncoder
# import urllib.request # to get files from web w/o !wget
# from shutil import unpack_archive # to unzip

# Concatenated Input Model

In [ ]:
# topology and good article here
# https://blog.goodaudience.com/introduction-to-1d-convolutional-neural-networks-in-keras-for-time-sequences-3a7ff801a2cf

description = '1D_CNN_Dense.py Concat input, 1D CNNs, with Dense Layers'

# Model params
num_filters = 50
k1_size = 35
k2_size = 9
dropout = 0.5
mpool_size = 8

# Training params
BATCH_SIZE = 128
MAX_EPOCHS = 200
es_patience = 20 # early stopping patience
lrn_rate = 0.0001 # learning rate, keras default is 0.001
lrn_verbose = 1 ##0 = silent, 1 = progress bar, 2 = one line per epoch

def build_model(x_train, y_train):
    """Builds 1D CNN followed by dense layer model.
    See .py for params.
    Returns: Model (Keras)"""
    n_timesteps, n_features, n_outputs = x_train.shape[1], x_train.shape[2], y_train.shape[1]
    model = keras.Sequential(
    [
        layers.Input(shape=x_train[0].shape),
        layers.BatchNormalization(scale=False),
        layers.Conv1D(filters=num_filters, kernel_size=k1_size, activation='relu',input_shape=(n_timesteps,n_features)),
        layers.Dropout(dropout),
        layers.Conv1D(filters=num_filters, kernel_size=k2_size, activation='relu'),
        layers.Dropout(dropout),
        layers.MaxPooling1D(pool_size=mpool_size),
        #layers.GlobalAveragePooling1D(),
        layers.Flatten(),
        layers.Dense(100, activation='relu'),
        layers.Dense(50, activation='relu'),
        layers.Dense(n_outputs, activation='softmax')
        ])
    return model 

def train_model(model,x_train, y_train, x_test, y_test):
    # see https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/EarlyStopping
    callback = EarlyStopping(monitor='val_loss', mode = 'min', patience=es_patience)
    opt = keras.optimizers.Adam(learning_rate=lrn_rate) # Keras default lr = 0.001
    model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

    history = model.fit(
        x_train,y_train,
        batch_size = BATCH_SIZE,
        epochs=MAX_EPOCHS,
        callbacks=[callback],
        validation_data=(x_test,y_test),
        verbose = lrn_verbose) #0 = silent, 1 = progress bar, 2 = one line per epoch
    return history, model

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization (BatchN  (None, 100, 4)           12        
 ormalization)                                                   
                                                                 
 conv1d (Conv1D)             (None, 66, 50)            7050      
                                                                 
 dropout (Dropout)           (None, 66, 50)            0         
                                                                 
 conv1d_1 (Conv1D)           (None, 58, 50)            22550     
                                                                 
 dropout_1 (Dropout)         (None, 58, 50)            0         
                                                                 
 max_pooling1d (MaxPooling1D  (None, 7, 50)            0         
 )                                                      